In [1]:
import matplotlib.pyplot as plt
plt.style.use('ggplot')
%matplotlib inline
%precision 4

import pyjags
import numpy as np

def summary(samples, varname, p=95, burnin=0, thin=1):
    values = samples[varname]
    (nb, iter, chains) = values.shape
    
    for i in range(nb):
        data = values[i][burnin::thin]

        ci = np.percentile(data, [100-p, p])
        
        print('{:<8}[{}] mean = {:>12.4f} (std = {:>10.4f}), {}% credible interval [{:>5.2f} {:>5.2f}]'.format(
          varname, i, np.mean(data), np.std(data), p, *ci))

In [2]:
sample = np.array([
        [10, 5, 12, 14, 30],
        [20, 14, 18, 22, 50],
        [20, 14, 18, 15, 25],
        [3, 2, 1, 2, 0]
    ])

cp, T = sample.shape

print(cp, T)

code = """
# bugs model for poisson distribs

model{
        for (i in 1:cp) {
            for (t in 1:T) {
                n[i, t] ~ dpois(a[i] * N[t])
            }
        }
        
        for (t in 1:T) {
            N[t]   ~ dpois(NN)
        }
        NN      ~ dunif(10,100)
        
        for (i in 1:cp) {
            b[i]   ~ dnorm(0, 10) I(0,)
        }
        
        for (i in 1:cp) {
            a[i]  <- b[i] / sum(b)
        }
}
"""

myvars=['NN', 'N', 'a']

%time model = pyjags.Model(code, data=dict(n=sample, cp=cp, T=T), chains=4, threads=4)
%time samples = model.sample(20000, vars=myvars)
samples.keys()

4 5
adapting: iterations 4000 of 4000, elapsed 0:00:00, remaining 0:00:00
CPU times: user 496 ms, sys: 28.4 ms, total: 525 ms
Wall time: 468 ms
sampling: iterations 1927 of 80000, elapsed 0:00:01, remaining 0:00:23
sampling: iterations 14067 of 80000, elapsed 0:00:01, remaining 0:00:05
sampling: iterations 21046 of 80000, elapsed 0:00:02, remaining 0:00:06
sampling: iterations 31119 of 80000, elapsed 0:00:03, remaining 0:00:04
sampling: iterations 39304 of 80000, elapsed 0:00:03, remaining 0:00:03
sampling: iterations 45517 of 80000, elapsed 0:00:04, remaining 0:00:03
sampling: iterations 55476 of 80000, elapsed 0:00:04, remaining 0:00:02
sampling: iterations 63750 of 80000, elapsed 0:00:05, remaining 0:00:01
sampling: iterations 68929 of 80000, elapsed 0:00:05, remaining 0:00:01
sampling: iterations 74606 of 80000, elapsed 0:00:06, remaining 0:00:00
sampling: iterations 80000 of 80000, elapsed 0:00:06, remaining 0:00:00
CPU times: user 8.69 s, sys: 665 ms, total: 9.36 s
Wall time: 6.2

dict_keys(['NN', 'N', 'a'])

In [3]:
burnin = 5000
        
for varname in myvars:
    summary(samples, varname, burnin=burnin, thin=4)

NN      [0] mean =      58.5322 (std =     4.7832), 95% credible interval [50.83 66.55]
N       [0] mean =      55.7821 (std =     5.7967), 95% credible interval [47.00 66.00]
N       [1] mean =      46.1269 (std =     5.5198), 95% credible interval [37.00 55.00]
N       [2] mean =      53.7649 (std =     5.7647), 95% credible interval [45.00 64.00]
N       [3] mean =      55.8573 (std =     5.8246), 95% credible interval [47.00 66.00]
N       [4] mean =      80.0723 (std =     6.5386), 95% credible interval [70.00 91.00]
a       [0] mean =       0.2418 (std =     0.0246), 95% credible interval [ 0.20  0.28]
a       [1] mean =       0.4167 (std =     0.0281), 95% credible interval [ 0.37  0.46]
a       [2] mean =       0.3110 (std =     0.0264), 95% credible interval [ 0.27  0.36]
a       [3] mean =       0.0304 (std =     0.0100), 95% credible interval [ 0.02  0.05]


## Now with some noise

In [24]:
sample = np.array([
        [10, 5, 22, 14, 130],
        [20, 14, 18, 22, 50],
        [20, 600, 18, 200, 25],
        [3, 2, 1, 7, 0],
        [500, 400, 1000, 300, 300],
        [60, 40, 20, 80, 800]
    ])

cp, T = sample.shape

print(cp, T)

code = """
# bugs model for poisson distribs

model{
        for (i in 1:cp) {
            for (t in 1:T) {
                m[i, t]       ~ dpois(a[i] * N[t])
                noise[i, t]   ~ dbern(p_noise)
                outl[i, t]    ~ dpois(N[t] * a[i] * outl_factor)
                mm[i, t, 1]  <- m[i, t]
                mm[i, t, 2]  <- m[i, t] + outl[i, t]
                
                n[i, t]       ~ dnorm(mm[i, t, choice[i, t]], tau)
                choice[i, t] <- noise[i, t] + 1
            }
        }
        
        tau         <- 1 / sigma / sigma
        sigma        ~ dunif(0.0001, 0.001)
        
        outl_factor  ~ dunif(1, 5)
        
        for (t in 1:T) {
            N[t]   ~ dpois(NNN)
        }
        
        NNN      ~ dunif(NN/2, NN*2)
        p_noise  ~ dbeta(1, 10)
        
        for (i in 1:cp) {
            b[i]   ~ dunif(0, 1)
        }
        
        for (i in 1:cp) {
            a[i]  <- b[i] / sum(b)
        }
}
"""

myvars=['NNN', 'N', 'a', 'p_noise', 'noise', 'sigma', 'outl_factor']

%time model = pyjags.Model(code,  \
                           data=dict(n=sample, cp=cp, T=T, NN=sample.sum(axis=0).mean()),  \
                           chains=4, threads=4, adapt=4000)
%time samples = model.sample(20000, vars=myvars)
samples.keys()

6 5
adapting: iterations 375 of 16000, elapsed 0:00:01, remaining 0:00:25
adapting: iterations 1411 of 16000, elapsed 0:00:01, remaining 0:00:14
adapting: iterations 3067 of 16000, elapsed 0:00:02, remaining 0:00:08
adapting: iterations 4168 of 16000, elapsed 0:00:02, remaining 0:00:07
adapting: iterations 5079 of 16000, elapsed 0:00:03, remaining 0:00:06
adapting: iterations 5997 of 16000, elapsed 0:00:04, remaining 0:00:06
adapting: iterations 6915 of 16000, elapsed 0:00:04, remaining 0:00:05
adapting: iterations 7799 of 16000, elapsed 0:00:05, remaining 0:00:05
adapting: iterations 9140 of 16000, elapsed 0:00:05, remaining 0:00:04
adapting: iterations 10238 of 16000, elapsed 0:00:06, remaining 0:00:03
adapting: iterations 11813 of 16000, elapsed 0:00:07, remaining 0:00:02
adapting: iterations 12971 of 16000, elapsed 0:00:07, remaining 0:00:02
adapting: iterations 14588 of 16000, elapsed 0:00:08, remaining 0:00:01
adapting: iterations 15931 of 16000, elapsed 0:00:08, remaining 0:00:0

dict_keys(['sigma', 'p_noise', 'NNN', 'noise', 'N', 'outl_factor', 'a'])

In [25]:
for varname in myvars:
    if varname != 'noise':
        summary(samples, varname, burnin=0, thin=4)

NNN     [0] mean =     791.0448 (std =    81.8339), 95% credible interval [709.40 942.39]
N       [0] mean =     699.0490 (std =    42.6579), 95% credible interval [645.00 781.00]
N       [1] mean =     893.6004 (std =    70.2541), 95% credible interval [787.00 1013.00]
N       [2] mean =     701.4340 (std =   176.5661), 95% credible interval [549.00 1021.00]
N       [3] mean =     704.1016 (std =    42.8683), 95% credible interval [650.00 786.00]
N       [4] mean =     955.6480 (std =   114.6564), 95% credible interval [818.00 1127.00]
a       [0] mean =       0.0459 (std =     0.0053), 95% credible interval [ 0.04  0.05]
a       [1] mean =       0.0316 (std =     0.0040), 95% credible interval [ 0.02  0.04]
a       [2] mean =       0.2036 (std =     0.0309), 95% credible interval [ 0.16  0.24]
a       [3] mean =       0.0035 (std =     0.0010), 95% credible interval [ 0.00  0.01]
a       [4] mean =       0.5083 (std =     0.0162), 95% credible interval [ 0.49  0.54]
a       [5] mean 

In [26]:
noise = samples["noise"]
(a, b, d, e) = noise.shape

noise.mean(axis=2).mean(axis=2)

array([[  0.0000e+00,   0.0000e+00,   0.0000e+00,   0.0000e+00,
          0.0000e+00],
       [  0.0000e+00,   0.0000e+00,   0.0000e+00,   0.0000e+00,
          0.0000e+00],
       [  0.0000e+00,   2.5000e-01,   0.0000e+00,   0.0000e+00,
          0.0000e+00],
       [  1.1250e-03,   4.8750e-04,   1.4750e-03,   1.0250e-03,
          3.3750e-04],
       [  0.0000e+00,   0.0000e+00,   7.5000e-01,   0.0000e+00,
          0.0000e+00],
       [  0.0000e+00,   0.0000e+00,   0.0000e+00,   0.0000e+00,
          5.0000e-01]])